# TF-IDF를 활용한 Passage Retrieval 실습

훈련하고자 하는 Corpus: korquad - 훈련 데이터셋

평가하고자 하는 Corpus: wikipedia 문서들

### Requirements

In [1]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==2.4.0 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.20.1 > /dev/null 2>&1

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


## 데이터셋 준비

KorQuAD train 데이터셋을 search corpus로 활용

In [3]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (C:\Users\AI_15\.cache\huggingface\datasets\squad_kor_v1\squad_kor_v1\1.0.0\18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용 (성능 향상을 위해 더 세밀한 토크나이저 활용 가능)

In [5]:
tokenizer_func = lambda x: x.split(' ')

In [6]:
tokenizer_func(corpus[0])[:10]

['발전소',
 '주변',
 '부지는',
 '에너지',
 '자립도시와',
 '과학공원으로',
 '계획되었으나',
 '‘시화호',
 '워터프론트’(시화나래)',
 '프로젝트']

## TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만들어보기 
(unigram, bigram 활용)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

In [8]:
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

c:\Users\AI_15\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
sp_matrix.shape

(9606, 1272768)

In [10]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

c:\Users\AI_15\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


               TF-IDF
인근           0.207740
유속을 느려지게     0.079626
연계될 예정이다.    0.079626
해수유통량을       0.079626
꼽힌다. 매월      0.079626
향후 개발되는      0.079626
일조하였으나 오염된   0.079626
착공하였다. 가물막이  0.079626
일조하였으나       0.079626
도입하여 2011년   0.079626


## TF-IDF embedding을 활용하여 passage retrieval 실습해보기

앞서 만든 sparse embedding을 활용하여, 실제 passage retrieval 수행해보기

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인해보기

In [11]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

Query를 tf-idf vector로 변환

In [12]:
query_vec = vectorizer.transform([query])

In [13]:
query_vec.shape

(1, 1272768)

변환된 query vector를 document들의 vector과 dot product를 수행 => Document들의 similarity ranking을 구함

In [14]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [15]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교해보기

In [16]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([2937, 2203, 5512], dtype=int32))

In [17]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")


[Search query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 

## Wikipedia documents에 대해 TF-IDF 실습하기

In [20]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

Cannot open cookies file 'C:/Users/AI_15/momo/python_prj/2022_AI_Expert/NLP_MRC/cookies.txt': No such file or directory
--2022-07-27 16:36:07--  https://docs.google.com/uc?export=download&confirm=$(wget%20--quiet%20--save-cookies%20/tmp/cookies.txt%20--keep-session-cookies%20--no-check-certificate%20'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd'%20-O-%20%7C%20sed%20-rn%20's/.*confirm=([0-9A-Za-z_]+).*/%5C1%5Cn/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd
Resolving docs.google.com (docs.google.com)... 142.250.207.14
Connecting to docs.google.com (docs.google.com)|142.250.207.14|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-07-27 16:36:07 ERROR 404: Not Found.



In [23]:
# First load wikipedia dump
import json

# TODO: Write your own path
dump_path = 'wikipedia_documents.json'
with open(dump_path, 'r', encoding="UTF-8") as f:
    wiki = json.load(f)

In [24]:
len(wiki), type(wiki)

(60613, dict)

In [25]:
wiki['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [26]:
# TODO: Generate corpus and Train wiki_vectorizer on wikipedia_documents.json

# wiki_matrix = transformed matrix of documents
corpus = [wiki[key]['text'] for key in wiki.keys()]
wiki_vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))
wiki_vectorizer.fit(corpus)
wiki_matrix = vectorizer.transform(corpus)

c:\Users\AI_15\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [27]:
# Answer: (60613, 8719114) (num_doc, num_entity)
wiki_matrix.shape

(60613, 1272768)

In [58]:
# TODO: transform query to sparse vector
#query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'
query = "대한민국의 행정부 수반은?"
query_vec = vectorizer.transform([query])

In [59]:
result = query_vec * wiki_matrix.T
result.shape

(1, 60613)

In [60]:
sorted_result= np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

In [61]:
k = 5
doc_scores[:k], doc_ids[:k]

(array([0.3962615 , 0.3962615 , 0.21010306, 0.14573027, 0.13200305]),
 array([ 2283,  6837, 34506, 56228, 57151], dtype=int32))

In [62]:
print("[Search query]\n", query, "\n")

for i in range(k):
    print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
    doc_id = doc_ids[i]
    print(corpus[doc_id], "\n")

[Search query]
 대한민국의 행정부 수반은? 

Top-1 passage with score 0.3963
이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 이들 국가들 중 상당수는 행정부 수반이 의회해산권을 갖고, 의회는 내각불신임권을 갖는데 이점 역시 내각제와 닮았다. 하지만 내각제와 달리 의회에서 선출되는 자는 행정부 수반의 지위 뿐만 아니라 국가 원수의 지위도 가진다. 그래서 의회에서 선출되는 자의 직위는 총리가 아니라, 대통령이다. 이처럼 1인이 행정부 수반과 국가 원수의 지위를 겸한다는 점에서 대통령중심제와 닮았다. 이상과 같은 이유로 이들 국가들의 정부 형태는 대통령제와 내각제의 절충형이지만, 행정 권한이 2인에게로 분리되어 있지 않으므로 이원집정부제는 아니다.

 
* 나우루
* 남아프리카 공화국
* 마셜 제도
* 마카오
* 미얀마
* 미크로네시아 연방
* 보츠와나
* 산마리노
* 수리남
* 키리바시
* 홍콩 

Top-2 passage with score 0.3963
이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 이들 국가들 중 상당수는 행정부 수반이 의회해산권을 갖고, 의회는 내각불신임권을 갖는데 이점 역시 내각제와 닮았다. 하지만 내각제와 달리 의회에서 선출되는 자는 행정부 수반의 지위 뿐만 아니라 국가 원수의 지위도 가진다. 그래서 의회에서 선출되는 자의 직위는 총리가 아니라, 대통령이다. 이처럼 1인이 행정부 수반과 국가 원수의 지위를 겸한다는 점에서 대통령중심제와 닮았다. 이상과 같은 이유로 이들 국가들의 정부 형태는 대통령제와 내각제의 절충형이지만, 행정 권한이 2인에게로 분리되어 있지 않으므로 이원집정부제는 아니다.

 
* 나우루
* 남아프리카 공화국
* 마셜 제도
* 마카오
* 미얀마
* 미크로네시아 연방
* 보츠와나
* 산마리노
* 수리남
* 키리바시
* 홍콩 

Top-3 passage with score 0.2101
1957년 조지타운 대학교를 역